# Example: Varying Injection Volume (Linearity)

Here outlines a basic linearity experiment where one sample (and therefore one vial location) is injected at varying injection volumes. 
* Varies injection volume within a range in a defined increment
* Samples in sample set defined in for loop, varying the injection volume

# Import

In [10]:
from OptiHPLCHandler import EmpowerHandler

handler = EmpowerHandler(
    project="WebAPI_test", address="XXXXXXXX", allow_login_without_context_manager=False
)

# Configuration of methods, nodes, systems and plates

Using get attributes to: obtain method from list_method, node from list_nodes, system from list_systems within defined node and define the plate type from list_plates.

In [60]:
with handler:
    list_methods = handler.GetMethodList()
    method = list_methods[-1]  # most recent method
    print(f"Method Set Name: {method}")

    list_nodes = handler.GetNodeNames()
    node = list_nodes[40]
    print(f"Node: {node}")

    list_systems = handler.GetSystemNames(node=node)
    system = list_systems[0]
    print(f"System Configuration Name: {system}")

    list_plates = handler.GetPlateTypeNames()[:6]
    plate = list_plates[5]
    print(f"Plate: {plate}")
    plates = {"1": plate, "2": plate}

C:\Users\slcb\Desktop\Programming\OptiHPLCHandler\src\OptiHPLCHandler\empower_api_core.py:172: UserWarning: The password will be sent in plain text.
  warnings.warn("The password will be sent in plain text.")


Method Set Name: B104_11_IClass_280nm_UPLC12
Node: Epdkhqr01044
System Configuration Name: 14_5208_HCLASS_BIN
Plate: ANSI-48Vial2mLHolder


Vary injection volume from a range of 1 to 3 µL in 1 µL increments

In [63]:
# Varying injection volume
sample_set_method_name = "test_samplesetmethod_0"

sample_list = []
list_inj = [i for i in range(1, 4, 1)]
# run linearity experiment from 1 to 3 µL in 1 µL increment

for i in range(len(list_inj)):
    sample_list.append(
        {
            "Method": method,
            "SamplePos": "1:A,1",  # specific vial selected
            "SampleName": f"Sample",
            "InjectionVolume": list_inj[i],
        },
    )

sample_list

[{'Method': 'B104_11_IClass_280nm_UPLC12',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 1},
 {'Method': 'B104_11_IClass_280nm_UPLC12',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 2},
 {'Method': 'B104_11_IClass_280nm_UPLC12',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 3}]

Add column conditioning and equilibration at start, thus selecting column position.

In [64]:
conditioning={
    "Function": {"member": "Condition Column"},
    "Method": method,
    "RunTime": 10,
    "ColumnPosition": {"member": "Position 3"}
}

equilibration={
    "Function": {"member": "Equilibrate"},
    "Method": method,
    "RunTime": 7,
    "ColumnPosition": {"member": "Position 3"}
}

sample_list.insert(0, conditioning)
sample_list.insert(1, equilibration)
sample_list

[{'Function': {'member': 'Condition Column'},
  'Method': 'B104_11_IClass_280nm_UPLC12',
  'RunTime': 10,
  'ColumnPosition': {'member': 'Position 3'}},
 {'Function': {'member': 'Equilibrate'},
  'Method': 'B104_11_IClass_280nm_UPLC12',
  'RunTime': 7,
  'ColumnPosition': {'member': 'Position 3'}},
 {'Method': 'B104_11_IClass_280nm_UPLC12',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 1},
 {'Method': 'B104_11_IClass_280nm_UPLC12',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 2},
 {'Method': 'B104_11_IClass_280nm_UPLC12',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 3}]

Post the experiment sample set method

In [ ]:
with handler:
    handler.PostExperiment(
        sample_set_method_name=sample_set_method_name,
        sample_list=sample_list,
        plates=plates,
        audit_trail_message=None,
    )

Run the sample set method

In [ ]:
with handler:
    handler.RunExperiment(
        sample_set_method=sample_set_method_name,
        sample_set_name=sample_set_method_name,
        system=system,
        node=node,
    )